In [79]:
import pandas as pd
import catboost 
import sklearn 
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool, cv
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import optuna

In [80]:
data = pd.read_csv('../data/processed/cleaned_data.csv', sep=',')
X = data.drop('Preis', axis=1)
y = data['Preis']

In [81]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [82]:
cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

train_pool = Pool(X_train, y_train, cat_features=cat_features)

In [83]:
# # mit bezeichnung und ausreiser
# best_params = {
#     'depth': 6,
#     'learning_rate': 0.25552573774921805,
#     'iterations': 598,
#     'l2_leaf_reg': 1.2219858759157394,
#     'bagging_temperature': 0.33587442182902427,
#     'random_strength': 3.1012551052358535,
#     'loss_function': 'RMSE',
#     'verbose': False
# }

# best_model = CatBoostRegressor(**best_params)
# best_model.fit(X_train, y_train, cat_features=cat_features)

# preds = best_model.predict(X_test)
# mse = mean_squared_error(y_test, preds)
# r2 = r2_score(y_test, preds)
# mean_absolute_error = mean_absolute_error(y_test, preds)
# print("Mean Absolute Error:", mean_absolute_error)
# print("Mean Squared Error:", mse)
# print("R2 Score:", r2)
# feature = best_model.get_feature_importance(prettified=True)
# print(feature)

In [84]:
# find best params
def objective(trial):
    params = {
        "depth": trial.suggest_int("depth", 6, 8),
        "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.3),
        "iterations": trial.suggest_int("iterations", 200, 800),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        "random_strength": trial.suggest_float("random_strength", 0, 5),
        "loss_function": "RMSE",
        "early_stopping_rounds": 50,
        "verbose": False
    }

    cv_results = cv(
            pool=train_pool,
            params=params,
            fold_count=8,                 
            shuffle=True,
            partition_random_seed=42,
            verbose=False
        )
    best_rmse = np.min(cv_results["test-RMSE-mean"])
    return best_rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Beste Hyperparameter:", study.best_params)

best_model = CatBoostRegressor(**study.best_params, silent=True)
best_model.fit(X_train, y_train, cat_features=cat_features)

preds = best_model.predict(X_test)
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
mean_absolute_error = mean_absolute_error(y_test, preds)

print("Mean Absolute Error:", mean_absolute_error)
print("Mean Squared Error:", mse)
print("R2 Score:", r2)
feature = best_model.get_feature_importance(prettified=True)
print(feature)


[I 2025-10-06 14:46:35,974] A new study created in memory with name: no-name-265097b3-7f7b-4b1b-b461-aebfa5ea0de6


Training on fold [0/8]

bestTest = 5.316891825
bestIteration = 337

Training on fold [1/8]

bestTest = 23.35623611
bestIteration = 340

Training on fold [2/8]

bestTest = 0.9627491113
bestIteration = 82

Training on fold [3/8]

bestTest = 2.541493144
bestIteration = 136

Training on fold [4/8]

bestTest = 15.22954815
bestIteration = 341

Training on fold [5/8]

bestTest = 2.45656654
bestIteration = 59

Training on fold [6/8]

bestTest = 3.631725827
bestIteration = 18

Training on fold [7/8]


[I 2025-10-06 14:47:27,991] Trial 0 finished with value: 7.798048283778 and parameters: {'depth': 8, 'learning_rate': 0.16764640684526882, 'iterations': 342, 'l2_leaf_reg': 9.531217197412479, 'bagging_temperature': 0.3233604383295704, 'random_strength': 4.841101279073893}. Best is trial 0 with value: 7.798048283778.



bestTest = 7.741667112
bestIteration = 341

Training on fold [0/8]

bestTest = 5.805631244
bestIteration = 203

Training on fold [1/8]

bestTest = 20.68319301
bestIteration = 205

Training on fold [2/8]

bestTest = 1.187818344
bestIteration = 16

Training on fold [3/8]

bestTest = 1.064458599
bestIteration = 91

Training on fold [4/8]

bestTest = 10.9524677
bestIteration = 202

Training on fold [5/8]

bestTest = 3.693087325
bestIteration = 205

Training on fold [6/8]

bestTest = 1.409247183
bestIteration = 175

Training on fold [7/8]


[I 2025-10-06 14:47:58,819] Trial 1 finished with value: 6.907338117925188 and parameters: {'depth': 6, 'learning_rate': 0.13717752798923316, 'iterations': 206, 'l2_leaf_reg': 5.02507728294577, 'bagging_temperature': 0.4484220407773418, 'random_strength': 0.11216005719139055}. Best is trial 1 with value: 6.907338117925188.


...


bestTest = 11.87372909
bestIteration = 385

Training on fold [0/8]

bestTest = 5.655599042
bestIteration = 272

Training on fold [1/8]

bestTest = 22.4402889
bestIteration = 272

Training on fold [2/8]

bestTest = 1.310071961
bestIteration = 18

Training on fold [3/8]

bestTest = 1.64580853
bestIteration = 53

Training on fold [4/8]

bestTest = 14.23696471
bestIteration = 271

Training on fold [5/8]

bestTest = 2.444935039
bestIteration = 112

Training on fold [6/8]

bestTest = 3.296618969
bestIteration = 271

Training on fold [7/8]


[I 2025-10-06 15:18:49,600] Trial 48 finished with value: 7.685126458301055 and parameters: {'depth': 6, 'learning_rate': 0.0597538671158369, 'iterations': 274, 'l2_leaf_reg': 5.317490240436941, 'bagging_temperature': 0.14377904644082157, 'random_strength': 4.242462022909907}. Best is trial 24 with value: 6.78415429902295.



bestTest = 9.986505066
bestIteration = 273

Training on fold [0/8]

bestTest = 6.453599339
bestIteration = 238

Training on fold [1/8]

bestTest = 23.98700729
bestIteration = 79

Training on fold [2/8]

bestTest = 1.142109082
bestIteration = 12

Training on fold [3/8]

bestTest = 1.840817708
bestIteration = 36

Training on fold [4/8]

bestTest = 16.29399467
bestIteration = 343

Training on fold [5/8]

bestTest = 3.943072972
bestIteration = 80

Training on fold [6/8]

bestTest = 3.046270168
bestIteration = 207

Training on fold [7/8]


[I 2025-10-06 15:20:02,202] Trial 49 finished with value: 8.55280145831194 and parameters: {'depth': 8, 'learning_rate': 0.15080178032107447, 'iterations': 419, 'l2_leaf_reg': 4.777963390686172, 'bagging_temperature': 0.07301909969822051, 'random_strength': 0.7593293767172657}. Best is trial 24 with value: 6.78415429902295.



bestTest = 10.43608544
bestIteration = 357

Beste Hyperparameter: {'depth': 6, 'learning_rate': 0.18298161321860076, 'iterations': 258, 'l2_leaf_reg': 4.918627265686312, 'bagging_temperature': 0.20116259123807867, 'random_strength': 0.5870492247152308}
Mean Absolute Error: 4.076170829386967
Mean Squared Error: 193.24419146473303
R2 Score: 0.49973965754669125
                              Feature Id  Importances
0                                Flaeche    48.147413
1                   Bauteilgewicht in kg    25.082560
2          Breite Laserzuschnitt in mm 2     7.150498
3                                 Dichte     5.662734
4                                Bereich     4.399512
5   Gesamtlänge aller Aussparungen in mm     3.335180
6                 Anzahl der Abkantungen     2.486888
7          Breite Laserzuschnitt in mm 1     2.414813
8                    Materialsärke in mm     0.793764
9                               Material     0.427329
10                          Bezugsquelle    